In [ ]:
from oxford_pets_train_script import *

In [ ]:
fs, ds, ts = gen_dataset()
to_pil = transforms.ToPILImage()
model = gen_model(fs.num_classes)
tdata, tlabel = ts[100]
tim = to_pil(tdata)

model.eval()
with torch.no_grad():
    preds = model([tdata])
    
plt.imshow(tim)